## Build a RAG System with Langchain and ChromaDB

In [89]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [90]:
# Langchain Imports

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.documents import Document

# vector stores
from langchain_community.vectorstores import Chroma

# Utility imports
import numpy as np
from typing import List

In [91]:
sample_docs = [
    """
    Machine Learning Fundamentals

    Machine learning is a subset of artificial intelligence that focuses on building systems that can learn from and make decisions based on data. It involves various algorithms and statistical models that enable computers to perform specific tasks without explicit instructions. The key types of machine learning include supervised learning, unsupervised learning, and reinforcement learning. Supervised learning uses labeled data to train models, while unsupervised learning finds patterns in unlabeled data. Reinforcement learning involves training agents to make sequences of decisions by rewarding desired behaviors. Applications of machine learning span across various industries, including healthcare, finance, marketing, and more. Common algorithms used in machine learning include decision trees, support vector machines, neural networks, and clustering techniques. As the field continues to evolve, advancements in deep learning and neural networks are driving significant progress in areas such as computer vision, natural language processing, and autonomous systems.
    """,

    """
    Natural Language Processing (NLP)

    Natural Language Processing (NLP) is a branch of artificial intelligence that focuses on the interaction between computers and human language. It involves the development of algorithms and models that enable machines to understand, interpret, and generate human language in a way that is meaningful. NLP encompasses various tasks such as text classification, sentiment analysis, named entity recognition, machine translation, and speech recognition. Techniques used in NLP include tokenization, part-of-speech tagging, syntactic parsing, and semantic analysis. Recent advancements in NLP have been driven by deep learning models such as transformers, which have significantly improved the performance of language models like BERT and GPT. Applications of NLP are widespread, including chatbots, virtual assistants, automated customer support, and content recommendation systems. As NLP continues to advance, it plays a crucial role in enhancing human-computer interaction and enabling more natural communication with technology.
    """,
    """
    Deep Learning and Neural Networks

    Deep learning is a subset of machine learning that focuses on using neural networks with multiple layers to model complex patterns in data. Neural networks are inspired by the structure and function of the human brain, consisting of interconnected nodes (neurons) that process information. Deep learning has revolutionized various fields, including computer vision, natural language processing, and speech recognition, by enabling the development of models that can automatically learn hierarchical representations of data. Key architectures in deep learning include convolutional neural networks (CNNs) for image processing, recurrent neural networks (RNNs) for sequential data, and transformers for language modeling. Training deep learning models typically requires large datasets and significant computational resources, often utilizing GPUs for efficient processing. The success of deep learning has led to breakthroughs in applications such as autonomous vehicles, medical image analysis, and real-time language translation. As research continues, deep learning is expected to drive further innovations in artificial intelligence and its applications across various domains.
    """
]

sample_docs

['\n    Machine Learning Fundamentals\n\n    Machine learning is a subset of artificial intelligence that focuses on building systems that can learn from and make decisions based on data. It involves various algorithms and statistical models that enable computers to perform specific tasks without explicit instructions. The key types of machine learning include supervised learning, unsupervised learning, and reinforcement learning. Supervised learning uses labeled data to train models, while unsupervised learning finds patterns in unlabeled data. Reinforcement learning involves training agents to make sequences of decisions by rewarding desired behaviors. Applications of machine learning span across various industries, including healthcare, finance, marketing, and more. Common algorithms used in machine learning include decision trees, support vector machines, neural networks, and clustering techniques. As the field continues to evolve, advancements in deep learning and neural networks 

In [92]:
### save sample documents to files
import tempfile
# temp_dir= tempfile.mkdtemp()

for i, doc in enumerate(sample_docs):
    # with open(os.path.join(temp_dir, f"doc_{i}.txt"), "w") as f:
    #     f.write(doc)
    with open(f"doc_{i}.txt", "w") as f:
        f.write(doc.strip())

print(f"Sample documents saved to root")

Sample documents saved to root


## Document Loading

In [93]:
from langchain_community.document_loaders import DirectoryLoader

loader = DirectoryLoader('.', glob='doc_*.txt', loader_cls=TextLoader, loader_kwargs={'encoding': 'utf-8'})

documents = loader.load()

print(f"Loaded {len(documents)} documents.")
print("First document content:")
print(documents[0].page_content[:200])  # print first 200 characters of the first document

Loaded 3 documents.
First document content:
Machine Learning Fundamentals

    Machine learning is a subset of artificial intelligence that focuses on building systems that can learn from and make decisions based on data. It involves various al


In [94]:
documents

[Document(metadata={'source': 'doc_0.txt'}, page_content='Machine Learning Fundamentals\n\n    Machine learning is a subset of artificial intelligence that focuses on building systems that can learn from and make decisions based on data. It involves various algorithms and statistical models that enable computers to perform specific tasks without explicit instructions. The key types of machine learning include supervised learning, unsupervised learning, and reinforcement learning. Supervised learning uses labeled data to train models, while unsupervised learning finds patterns in unlabeled data. Reinforcement learning involves training agents to make sequences of decisions by rewarding desired behaviors. Applications of machine learning span across various industries, including healthcare, finance, marketing, and more. Common algorithms used in machine learning include decision trees, support vector machines, neural networks, and clustering techniques. As the field continues to evolve, 

## Document Splitting

In [95]:
# Initialize Text Splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    length_function=len,
    separators=[" "] # Hierarchy of separators
)

chunks = text_splitter.split_documents(documents)

chunks

[Document(metadata={'source': 'doc_0.txt'}, page_content='Machine Learning Fundamentals\n\n    Machine learning is a subset of artificial intelligence that focuses on building systems that can learn from and make decisions based on data. It involves various algorithms and statistical models that enable computers to perform specific tasks without explicit instructions. The key types of machine learning include supervised learning, unsupervised learning, and reinforcement learning. Supervised learning uses labeled data to train models, while unsupervised'),
 Document(metadata={'source': 'doc_0.txt'}, page_content='labeled data to train models, while unsupervised learning finds patterns in unlabeled data. Reinforcement learning involves training agents to make sequences of decisions by rewarding desired behaviors. Applications of machine learning span across various industries, including healthcare, finance, marketing, and more. Common algorithms used in machine learning include decision 

## Initialize ChromaDB Vector store and store the chunks

In [96]:
## Create a ChromaDB Vector Store

persist_directory = './chroma_db'

## Initialize ChromaDB with HuggingFace Embeddings
vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2"),
    persist_directory=persist_directory,
    collection_name="rag_collection"
)

print(f"Vector store created with {vectorstore._collection.count()} vectors.")
print(f"Vectors persisted in directory: {persist_directory}")

Vector store created with 45 vectors.
Vectors persisted in directory: ./chroma_db


### Test Similarity search

In [97]:
query = "What are types of machine learning?"

similar_docs = vectorstore.similarity_search(query, k=3)

similar_docs

[Document(metadata={'source': 'doc_0.txt'}, page_content='Machine Learning Fundamentals\n\n    Machine learning is a subset of artificial intelligence that focuses on building systems that can learn from and make decisions based on data. It involves various algorithms and statistical models that enable computers to perform specific tasks without explicit instructions. The key types of machine learning include supervised learning, unsupervised learning, and reinforcement learning. Supervised learning uses labeled data to train models, while unsupervised'),
 Document(metadata={'source': 'doc_0.txt'}, page_content='Machine Learning Fundamentals\n\n    Machine learning is a subset of artificial intelligence that focuses on building systems that can learn from and make decisions based on data. It involves various algorithms and statistical models that enable computers to perform specific tasks without explicit instructions. The key types of machine learning include supervised learning, unsu

In [98]:
query = "What is NLP?"

similar_docs = vectorstore.similarity_search(query, k=3)

similar_docs

[Document(metadata={'source': 'doc_1.txt'}, page_content='Natural Language Processing (NLP)\n\n    Natural Language Processing (NLP) is a branch of artificial intelligence that focuses on the interaction between computers and human language. It involves the development of algorithms and models that enable machines to understand, interpret, and generate human language in a way that is meaningful. NLP encompasses various tasks such as text classification, sentiment analysis, named entity recognition, machine translation, and speech recognition. Techniques'),
 Document(metadata={'source': 'doc_1.txt'}, page_content='Natural Language Processing (NLP)\n\n    Natural Language Processing (NLP) is a branch of artificial intelligence that focuses on the interaction between computers and human language. It involves the development of algorithms and models that enable machines to understand, interpret, and generate human language in a way that is meaningful. NLP encompasses various tasks such as 

In [99]:
query = "What is deep learning?"

similar_docs = vectorstore.similarity_search(query, k=3)

similar_docs

[Document(metadata={'source': 'doc_2.txt'}, page_content='Deep Learning and Neural Networks\n\n    Deep learning is a subset of machine learning that focuses on using neural networks with multiple layers to model complex patterns in data. Neural networks are inspired by the structure and function of the human brain, consisting of interconnected nodes (neurons) that process information. Deep learning has revolutionized various fields, including computer vision, natural language processing, and speech recognition, by enabling the development of models that'),
 Document(metadata={'source': 'doc_2.txt'}, page_content='Deep Learning and Neural Networks\n\n    Deep learning is a subset of machine learning that focuses on using neural networks with multiple layers to model complex patterns in data. Neural networks are inspired by the structure and function of the human brain, consisting of interconnected nodes (neurons) that process information. Deep learning has revolutionized various fields

### Advanced Similarity Search with Scores

In [100]:
result_scores= vectorstore.similarity_search_with_score(query, k=3)
result_scores

[(Document(metadata={'source': 'doc_2.txt'}, page_content='Deep Learning and Neural Networks\n\n    Deep learning is a subset of machine learning that focuses on using neural networks with multiple layers to model complex patterns in data. Neural networks are inspired by the structure and function of the human brain, consisting of interconnected nodes (neurons) that process information. Deep learning has revolutionized various fields, including computer vision, natural language processing, and speech recognition, by enabling the development of models that'),
  0.5512487888336182),
 (Document(metadata={'source': 'doc_2.txt'}, page_content='Deep Learning and Neural Networks\n\n    Deep learning is a subset of machine learning that focuses on using neural networks with multiple layers to model complex patterns in data. Neural networks are inspired by the structure and function of the human brain, consisting of interconnected nodes (neurons) that process information. Deep learning has revo

### Understanding Similarity Scores

The similarity score represents how closely related a document chunk is to your query, The scoring depends on the distance metric used:

ChromaDB by default uses L2 distance (Euclidean distance)
- Lower scores =  MORE similar (closer in vector space)
- Score of 0 =  identical vectors
- Typical range: 0 to 2 (but can be higher)

Cosine similarity (if configured):

- Higher scores =  MORE similar
- Range: -1 to 1 (1 being identical)

## Initialize LLM, RAG Chain, Prompt Tempate, Query the RAG system

In [101]:
from langchain_ollama import ChatOllama

llm = ChatOllama(model="mistral:latest")

In [102]:
test_response =  llm.invoke("What is Large Language Models?")

test_response

AIMessage(content=" Large Language Models (LLMs) are a type of artificial intelligence designed to process and generate human-like text based on the data they were trained on. They can be used for various tasks such as answering questions, writing essays, summarizing texts, translating languages, and even generating creative content like stories or poems. These models learn from vast amounts of text data during their training process, allowing them to understand and mimic the way humans communicate in written form.\n\nNotable examples include ChatGPT (developed by OpenAI), BERT (Google), and T5 (Carnegie Mellon University). They are typically trained on a diverse range of internet text, such as books, articles, websites, and other written content to help them learn the nuances of language usage. However, it's important to note that while these models can generate human-like text, they don't truly understand or have consciousness – they simply analyze patterns in their training data and

In [103]:
# Alternative waqy to initialize chat models

from langchain.chat_models.base import init_chat_model

chat_model = init_chat_model("ollama:mistral:latest")
#llm = init_chat_model("groq:")

chat_model

ChatOllama(model='mistral:latest')

In [104]:
chat_model.invoke("Explain the concept of reinforcement learning.")

AIMessage(content=" Reinforcement Learning (RL) is a type of machine learning where an agent learns to make decisions by taking actions in an environment to achieve a goal. The agent receives rewards or penalties (reinforcements) for its actions, and the goal is to learn a policy that maximizes the total reward over time.\n\nHere's a breakdown of key components:\n\n1. **Agent**: The entity that perceives its environment through sensors and acts upon it through actuators. It learns by interacting with the environment and adjusting its behavior based on the rewards received.\n\n2. **Environment**: A set of possible states, actions, and a transition function that describes how the environment moves from one state to another when an action is taken. The environment also provides rewards or penalties (reinforcements) for each action.\n\n3. **Policy**: A strategy that outlines what action the agent should take in any given state of the environment. The policy can be deterministic (always cho

## Modern RAG Chain

In [105]:
from langchain_classic.chains.retrieval import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_classic.chains.combine_documents.stuff import create_stuff_documents_chain

In [106]:
## Convert vectorstore to a retriever

retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}
)

retriever

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x0000024DFF6D1BD0>, search_kwargs={'k': 3})

In [107]:
##Create Prompt Template
system_prompt = """You are an assistant for question-answering tasks.
Use the context provided to answer the question as accurately as possible.
If the context does not contain the answer, respond with 'I don't know'.
Use three sentences maximum to answer the question and keep it concise.

Context: {context}"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}")
])

In [108]:
prompt

ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks.\nUse the context provided to answer the question as accurately as possible.\nIf the context does not contain the answer, respond with 'I don't know'.\nUse three sentences maximum to answer the question and keep it concise.\n\nContext: {context}"), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])

In [109]:
## Create Documents Chain
documents_chain = create_stuff_documents_chain(
    llm=chat_model,
    prompt=prompt
)

documents_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks.\nUse the context provided to answer the question as accurately as possible.\nIf the context does not contain the answer, respond with 'I don't know'.\nUse three sentences maximum to answer the question and keep it concise.\n\nContext: {context}"), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])
| ChatOllama(model='mistral:latest')
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_document

This chain:

- Takes retrieved documents
- "Stuffs" them into prompt's {context} placeholder.
- Sends the complete prompt to the LLM.
- Returns the LLM's response.

In [110]:
### Create the Final RAG Chain
rag_chain = create_retrieval_chain(retriever,documents_chain)
rag_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x0000024DFF6D1BD0>, search_kwargs={'k': 3}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks.\nUse the context provided to answer the question as accurately as possible

Using this create_retrieval_chain we are able to another retreval layer to our stuff document chain which uses the retriever to fetch relavant documents from our vectorstore and sends it to the stuff chain which sends it to the LLM.

In [111]:
response = rag_chain.invoke({"input": "Explain the different types of machine learning."})

response

{'input': 'Explain the different types of machine learning.',
 'context': [Document(metadata={'source': 'doc_0.txt'}, page_content='Machine Learning Fundamentals\n\n    Machine learning is a subset of artificial intelligence that focuses on building systems that can learn from and make decisions based on data. It involves various algorithms and statistical models that enable computers to perform specific tasks without explicit instructions. The key types of machine learning include supervised learning, unsupervised learning, and reinforcement learning. Supervised learning uses labeled data to train models, while unsupervised'),
  Document(metadata={'source': 'doc_0.txt'}, page_content='Machine Learning Fundamentals\n\n    Machine learning is a subset of artificial intelligence that focuses on building systems that can learn from and make decisions based on data. It involves various algorithms and statistical models that enable computers to perform specific tasks without explicit instru

In [112]:
response["answer"]

' Machine Learning has three primary categories:\n1. Supervised Learning: This type of machine learning involves training models using labeled data, where both input and output data are provided during training for accurate predictions. Examples include linear regression and support vector machines (SVM).\n2. Unsupervised Learning: In contrast to supervised learning, unsupervised learning trains models on unlabeled data without specific outputs. The algorithms aim to identify patterns or structure in the input data. Clustering and dimensionality reduction are examples of unsupervised learning.\n3. Reinforcement Learning: This approach involves an agent interacting with its environment to learn through trial-and-error. The agent receives rewards or penalties based on actions, and it learns to take actions that maximize long-term rewards. Examples include Q-learning and deep reinforcement learning.'

In [113]:
# Function to query the modern RAG system

def query_rag_modern(question: str) -> str:
    print(f"Question: {question}")
    print("-"*50)

    # Using create_retrieval_chain approach
    result = rag_chain.invoke({"input": question})

    print(f"Answer: {result}")
    print("\nRetrieved Context:")
    for i, doc in enumerate(result['context']):
        print(f"\n ----- Source {i+1} -----")
        print(doc.page_content[:200]+"...")

    return result['answer']

test_questions = [
    "What are three types of machine learning?",
    "What is Deep Learning and how does it relate to Neural Networks?",
    "What are CNNs used for?"
]

for question in test_questions:
    query_rag_modern(question)
    print("\n"+"="*80+"\n")

Question: What are three types of machine learning?
--------------------------------------------------
Answer: {'input': 'What are three types of machine learning?', 'context': [Document(metadata={'source': 'doc_0.txt'}, page_content='Machine Learning Fundamentals\n\n    Machine learning is a subset of artificial intelligence that focuses on building systems that can learn from and make decisions based on data. It involves various algorithms and statistical models that enable computers to perform specific tasks without explicit instructions. The key types of machine learning include supervised learning, unsupervised learning, and reinforcement learning. Supervised learning uses labeled data to train models, while unsupervised'), Document(metadata={'source': 'doc_0.txt'}, page_content='Machine Learning Fundamentals\n\n    Machine learning is a subset of artificial intelligence that focuses on building systems that can learn from and make decisions based on data. It involves various algo

# Creating Rag Chain Alternative - Using LCEL

In [114]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate


In [115]:
#Create a custom Prompt
custom_prompt = ChatPromptTemplate.from_template(
    """Use the context provided to answer the question as accurately as possible.
If the context does not contain the answer, respond with 'I don't know'.
Use three sentences maximum to answer the question and keep it concise.

Context: {context}

Question: {question}

Answer:"""
)

custom_prompt


ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="Use the context provided to answer the question as accurately as possible.\nIf the context does not contain the answer, respond with 'I don't know'.\nUse three sentences maximum to answer the question and keep it concise.\n\nContext: {context}\n\nQuestion: {question}\n\nAnswer:"), additional_kwargs={})])

In [116]:
retriever

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x0000024DFF6D1BD0>, search_kwargs={'k': 3})

In [117]:
## Format the output documents for the prompt
def format_docs(docs: List[Document]) -> str:
    return "\n\n".join([doc.page_content for doc in docs])

In [118]:
## Build the RAG Chain using LCEL
rag_chain_lcel = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough()  # we use runnable passthrough to accept question input during runtime
    }
    | custom_prompt
    | chat_model
    | StrOutputParser()
)

rag_chain_lcel

{
  context: VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x0000024DFF6D1BD0>, search_kwargs={'k': 3})
           | RunnableLambda(format_docs),
  question: RunnablePassthrough()
}
| ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="Use the context provided to answer the question as accurately as possible.\nIf the context does not contain the answer, respond with 'I don't know'.\nUse three sentences maximum to answer the question and keep it concise.\n\nContext: {context}\n\nQuestion: {question}\n\nAnswer:"), additional_kwargs={})])
| ChatOllama(model='mistral:latest')
| StrOutputParser()

In [121]:
response = rag_chain_lcel.invoke("Explain the different types of machine learning.")
response

" Machine Learning involves three primary types:\n1. Supervised Learning: This type uses labeled data for training models. The algorithm learns from input-output pairs and aims to predict outputs for new inputs.\n2. Unsupervised Learning: In this method, the algorithm learns patterns in unlabeled data without explicit guidance on what to look for. It's often used for clustering or dimensionality reduction.\n3. Reinforcement Learning: This type involves an agent learning through trial and error by interacting with its environment. The agent receives rewards or penalties for its actions, helping it learn the best action in a given situation."

In [141]:
context = retriever.invoke("Explain the different types of machine learning.")
context

[Document(metadata={'source': 'doc_0.txt'}, page_content='Machine Learning Fundamentals\n\n    Machine learning is a subset of artificial intelligence that focuses on building systems that can learn from and make decisions based on data. It involves various algorithms and statistical models that enable computers to perform specific tasks without explicit instructions. The key types of machine learning include supervised learning, unsupervised learning, and reinforcement learning. Supervised learning uses labeled data to train models, while unsupervised'),
 Document(metadata={'source': 'doc_0.txt'}, page_content='Machine Learning Fundamentals\n\n    Machine learning is a subset of artificial intelligence that focuses on building systems that can learn from and make decisions based on data. It involves various algorithms and statistical models that enable computers to perform specific tasks without explicit instructions. The key types of machine learning include supervised learning, unsu

In [139]:
# Query using the LCEL RAG Chain

def query_rag_lcel(question: str) -> str:
    print(f"Question: {question}")
    print("-"*50)

    # Using create_retrieval_chain approach
    result = rag_chain_lcel.invoke(question)

    print(f"Answer: {result}")
    
    docs = retriever.invoke(question)
    print("\nSource Context:")

    for i, doc in enumerate(docs):
        print(f"\n ----- Source {i+1} -----")
        print(doc.page_content[:200]+"...")


test_questions = [
    "What are three types of machine learning?",
    "What is Deep Learning and how does it relate to Neural Networks?",
    "What are CNNs used for?"
]


In [140]:
for question in test_questions:
    query_rag_lcel(question)
    print("\n"+"="*80+"\n")

Question: What are three types of machine learning?
--------------------------------------------------
Answer:  The three main types of machine learning are:
1. Supervised Learning
2. Unsupervised Learning
3. Reinforcement Learning

Source Context:

 ----- Source 1 -----
Machine Learning Fundamentals

    Machine learning is a subset of artificial intelligence that focuses on building systems that can learn from and make decisions based on data. It involves various al...

 ----- Source 2 -----
Machine Learning Fundamentals

    Machine learning is a subset of artificial intelligence that focuses on building systems that can learn from and make decisions based on data. It involves various al...

 ----- Source 3 -----
Machine Learning Fundamentals

    Machine learning is a subset of artificial intelligence that focuses on building systems that can learn from and make decisions based on data. It involves various al...


Question: What is Deep Learning and how does it relate to Neural Net